# Loading helper libraries 

In [36]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import re
import ast
import patsy
import datetime
import math

from pathlib import Path
from patsy import dmatrices
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from plotnine import *
from stargazer import stargazer
from statsmodels.tools.eval_measures import mse,rmse
from collections import defaultdict
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from plotnine.data import mpg
from mizani.formatters import percent_format



In [37]:
path = Path(os.getcwd())
tech_prep = os.path.join(str(path), "utils")
sys.path.append(tech_prep)
from py_helper_functions import *

# Loading Dataset 

### 1. main_df is the main dataset for my city, Lisbon.
### 2. amenities_cleaned_df is the dataset used for mapping amenities to my definitons

In [38]:
main_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/lisbon_listings.csv', delimiter=",",encoding="utf-8")
amenities_cleaned_df = pd.read_csv('https://raw.githubusercontent.com/arbash-malik/DA3/main/Assignment%202/unique_amenities_cleaned.csv', delimiter=",",encoding="utf-8")


In [39]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22605 entries, 0 to 22604
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            22605 non-null  int64  
 1   listing_url                                   22605 non-null  object 
 2   scrape_id                                     22605 non-null  int64  
 3   last_scraped                                  22605 non-null  object 
 4   source                                        22605 non-null  object 
 5   name                                          22605 non-null  object 
 6   description                                   22586 non-null  object 
 7   neighborhood_overview                         13429 non-null  object 
 8   picture_url                                   22605 non-null  object 
 9   host_id                                       22605 non-null 

#### Since our client  will be operating small and mid-size apartments hosting 2-6 guests. It makes sense to filter out listings that are a room or a shared unit and only keep the ones that a customer can book entirely

In [40]:
main_df = main_df[main_df["property_type"].str.contains(r'^Entire', regex=True)] # fitlering for entire property types
main_df = main_df.loc[(main_df['accommodates'] >= 2) & (main_df['accommodates'] <= 6)] # fitlering for 2-6 accommodates

In [41]:
main_df.head().T

,0,1,2,3,5
id,6499,25659,119120,29248,29396
listing_url,https://www.airbnb.com/rooms/6499,https://www.airbnb.com/rooms/25659,https://www.airbnb.com/rooms/119120,https://www.airbnb.com/rooms/29248,https://www.airbnb.com/rooms/29396
scrape_id,20230911050651,20230911050651,20230911050651,20230911050651,20230911050651
last_scraped,2023-09-11,2023-09-12,2023-09-12,2023-09-12,2023-09-12
source,city scrape,city scrape,city scrape,city scrape,city scrape
...,...,...,...,...,...
calculated_host_listings_count,1,1,8,5,1
calculated_host_listings_count_entire_homes,1,1,8,5,1
calculated_host_listings_count_private_rooms,0,0,0,0,0
calculated_host_listings_count_shared_rooms,0,0,0,0,0
